In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

import requests
import xml.etree.ElementTree as ET
import pandas as pd
import os
import json
import string
import requests
import xmltodict
import datetime
import base64
import datetime as dt

from dotenv import load_dotenv
load_dotenv()


In [ ]:
# To call the get_results function and execute the Finding API, provide the APPLICATION_ID and payload as arguments
api_key = os.environ.get('EBAY_API_KEY')

# To execute the Trading API, provide the APPLICATION_ID, DEVELOPER_ID, CERTIFICATION_ID and payload as arguments
cert_id = os.environ.get('EBAY_CERT_ID')
dev_id = os.environ.get('EBAY_DEV_ID')

user_token = os.environ.get('USER_TOKEN')

# eBay API credentials
endpoint = 'https://api.ebay.com/ws/api.dll'

# Define API headers
headers = {
    'X-EBAY-API-CALL-NAME': 'GetOrders',
    'X-EBAY-API-COMPATIBILITY-LEVEL': '967',
    'X-EBAY-API-DEV-NAME': dev_id,
    'X-EBAY-API-APP-NAME': api_key,
    'X-EBAY-API-CERT-NAME': cert_id,
    'X-EBAY-API-SITEID': '0',
    'Content-Type': 'text/xml'
}

In [ ]:


def get_orders():

      # Set the request fields
    fields = {
        'DetailLevel': 'ReturnAll',
        'OrderStatus': 'Completed',
        'CreateTimeFrom': dt.datetime.now() - dt.timedelta(days=300),
        'CreateTimeTo': dt.datetime.now(),
        'OrderRole': 'Seller',
        'Pagination': {
            'EntriesPerPage': 100,
            'PageNumber': 1
        },
        'OrderStatusFilter': 'Paid'
    }
    
    # Define API request payload
    payload = f'''
    <?xml version="1.0" encoding="utf-8"?>
    <GetOrdersRequest xmlns="urn:ebay:apis:eBLBaseComponents">
      <RequesterCredentials>
        <eBayAuthToken>{user_token}</eBayAuthToken>
      </RequesterCredentials>
      <ModTimeFrom>2022-01-01T00:00:00.000Z</ModTimeFrom>
      <ModTimeTo>2023-06-21T23:59:59.999Z</ModTimeTo>
    '''
    
    # Add fields to the payload
    for key, value in fields.items():
        if isinstance(value, dict):
            for inner_key, inner_value in value.items():
                payload += f'\n  <{key}><{inner_key}>{inner_value}</{inner_key}></{key}>'
        else:
            payload += f'\n  <{key}>{value}</{key}>'
    
    # Close the payload XML tag
    payload += '\n</GetOrdersRequest>'

    response = requests.post(endpoint, headers=headers, json=payload)


    # Parse the response and get the order information
    orders = response.reply.OrderArray.Order

    sales_data = []
    for order in orders:
        try:
            payment_status = order.MonetaryDetails.Payments.Payment.PaymentStatus
            ebay_payment_status = order.CheckoutStatus.eBayPaymentStatus
            order_id = order.OrderID
            item_id = order.TransactionArray.Transaction[0].Item.ItemID
            item_response = trading_api.execute('GetItem', {'ItemID': item_id})
            category_name = item_response.reply.Item.PrimaryCategory.CategoryName
            title = order.TransactionArray.Transaction[0].Item.Title
            quantity_purchased = order.TransactionArray.Transaction[0].QuantityPurchased
            buyer_username = order.BuyerUserID
            buyer_full_name = order.ShippingAddress.Name
            buyer_address_line1 = order.ShippingAddress.Street1
            buyer_address_line2 = order.ShippingAddress.Street2
            buyer_city = order.ShippingAddress.CityName
            buyer_state = order.ShippingAddress.StateOrProvince
            buyer_postal_code = order.ShippingAddress.PostalCode
            buyer_country = order.ShippingAddress.CountryName
            item_price = order.Total
            order_date = order.CreatedTime

            # Append the order data to the sales_data list
            sales_data.append({
                'PaymentStatus': payment_status,
                'eBayPaymentStatus': ebay_payment_status,
                'OrderID': order_id,
                'ItemID': item_id,
                'CategoryName': category_name,
                'Title': title,
                'QuantityPurchased': quantity_purchased,
                'BuyerUsername': buyer_username,
                'BuyerFullName': buyer_full_name,
                'BuyerAddressLine1': buyer_address_line1,
                'BuyerAddressLine2': buyer_address_line2,
                'BuyerCity': buyer_city,
                'BuyerState': buyer_state,
                'BuyerPostalCode': buyer_postal_code,
                'BuyerCountry': buyer_country,
                'ItemPrice': item_price,
                'OrderDate': order_date
            })

        except Exception as e:
            print(f"This is the error: {e}")
            print("Moving on...")
            continue

    # Create a DataFrame from the sales_data list
    df = pd.DataFrame(sales_data)

    # Fill missing fields with 'N/A'
    df.fillna('N/A', inplace=True)

    #print(df)
    return df

sales_report = get_orders()
print(sales_report)

In [ ]:
# Path to the CSV file
csv_file = r'C:\Users\Samuel Coromandel\Ebay Store Sales Dashboard\Ebay Store\eBay-ListingsSalesReport.csv'

# Read the CSV file with the first row as column headers
salesreport_df = pd.read_csv(csv_file, header=7)

# Set display options to show all columns without truncation
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)

# drop useless column data
columns_to_remove = ["Shipping and handling paid by buyer to you", "Insertion fees", "Optional listing upgrade fees", "Ads Standard fees", "Ads Advanced fees", "Ads Express fees", "International fees", "Other eBay fees", "Deposit processing fees", "Taxes and government fees paid by buyer to you", "Shipping labels cost (Amount you paid to buy shipping labels on eBay)", "Quantity sold via promoted listing", "Quantity sold via Seller Initiated Offers"]
salesreport_df.drop(columns=columns_to_remove, inplace=True)

# Print the DataFrame with the updated column headers
print(salesreport_df)

In [ ]:
purchasehistory_df = pd.read_csv(r'C:\Users\Samuel Coromandel\Ebay Store Sales Dashboard\Ebay Store\COGS.csv')
#purchasehistory_df.drop(columns=['OrderNotes', 'TrackingNumber', 'View Order Detail'], inplace=True)
print(purchasehistory_df)

In [ ]:
otherpurchases_df = pd.read_csv(r'C:\Users\Samuel Coromandel\Ebay Store Sales Dashboard\Ebay Store\COGS_other.csv', encoding='latin1')
#purchasehistory_df.drop(columns=['OrderNotes', 'TrackingNumber', 'View Order Detail'], inplace=True)
print(otherpurchases_df)

In [ ]:
#sql logic: salesreport_df as sr join purchasehistory_df as ph on sr.'Listing title' = ph.'ItemName'

# Problem: Ideally perform the join on 'Listing title' and 'ItemName' columns, however, some title & names don't match
# Solution 1: spend time writing a complex script to match vaguely matching strings
# Solution 2: use my custom ebay app to get the matching dictionary key-value pairs for item numbers (time consuming)
# Solution 3: Don't use the conjoined data to analyze metrics. create a new column based on custom formula for the metrics needed 

# Joining the data for reference but it's not usable in the current state.  
# logic: pd.concat([df1,df2], axis=1)
# Concatenate the DataFrames vertically to stack the values on top of each other
join_df = pd.concat([purchasehistory_df, otherpurchases_df], ignore_index=True)

# Print the joined DataFrame
print(join_df.tail(20))

## Exploratory Data Analysis

In [ ]:
def check_data(dataframe, head=5):
    print(" SHAPE ".center(70,'-'))
    print('Rows: {}'.format(dataframe.shape[0]))
    print('Columns: {}'.format(dataframe.shape[1]))
    print(" TYPES ".center(70,'-'))
    print(dataframe.dtypes)
    print(" HEAD ".center(70,'-'))
    print(dataframe.head(head))
    print(" TAIL ".center(70,'-'))
    print(dataframe.tail(head))
    print(" MISSING VALUES ".center(70,'-'))
    print(dataframe.isnull().sum())
    print(" DUPLICATED VALUES ".center(70,'-'))
    print(dataframe.duplicated().sum())
    print(" QUANTILES ".center(70,'-'))
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)
    
check_data(join_df)

In [ ]:
drop_rows = ["Heavy Duty Micro USB Fast Charger Data Cable Cord For Samsung Android HTC LG US",
        "Memory Card 32GB 64GB 128GB SDXC/SDHC Mini sd card class 10 TF Flash 8GB 16GB Mi",
        "Exercise Bike Home Gym  Indoor Cycling Bike Stationary Bicycle Cardio Workout",
        "LG V60 ThinQ 5G LMV600AM GSM Unlocked (AT&T/T-mobile) 128GB Classy Blue 6.8\""]

join_df["ItemName"] = join_df["ItemName"].apply(
    lambda x: "drop_this" if any(x in y for y in drop_rows) else x
)
#print(join_df[join_df["ItemName"]== "drop_this"])

df = join_df.drop(join_df[join_df["ItemName"] == "drop_this"].index)

print(df.sort_values("ItemPrice", ascending=False))

In [ ]:
#Clean up the Item sales column using pandas string methods
salesreport_df['Item sales'] = salesreport_df['Item sales'].str.replace('$', '').str.strip().astype(float)


print(salesreport_df['Item sales'].dtype)


Descriptive Statistics

In [ ]:
df.describe()

In [ ]:
df.to_csv('COGS')
salesreport_df.to_csv('sales_report')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def binomial_distribution(n, p):
    x = np.arange(0, n+1)
    probabilities = [binomial_pmf(k, n, p) for k in x]

    plt.bar(x, probabilities)
    plt.xlabel('Number of Successes')
    plt.ylabel('Probability')
    plt.title('Binomial Distribution (n={}, p={})'.format(n, p))
    plt.show()

def binomial_pmf(k, n, p):
    binomial_coeff = np.math.comb(n, k)
    probability = binomial_coeff * (p ** k) * ((1 - p) ** (n - k))
    return probability

# Example usage
n = 10  # Number of trials
p = 0.5  # Probability of success

binomial_distribution(n, p)

In [ ]:
Gross Profit Margin = sales_report["Net sales (Net of taxes and selling costs)"] - COGS["ItemPrice"]

#### Scratch work (converting DAX logic into python and vice-versa): 
NetSales_clean = IF(ISBLANK(Joined_Ebay_Store_Data[Net sales (Net of taxes and selling costs)]), Joined_Ebay_Store_Data[ItemPrice]*1.2, Joined_Ebay_Store_Data[Net sales (Net of taxes and selling costs)])
Month = Joined_Ebay_Store_Data[OrderDate]


Next year: Create a store-itemid dictionary that checks the listing status of the new itemid as completed as sold for your store and searches for the itemid of the original item that your item is based on so that it keeps a separate dictionary of all sold items. Then when joining the sales_report table with the purchase_history table, you can join on where salesreport_df.merge(purchasehistory_df, on/where=store_items[salesreport_df["eBay item ID"]]["original_id"]=purchasehistory_df["Item ID"]). This is joining the tables based on the original item for the ebay item id sold and the item id that was actual purchased which should be the original item id.